In [1]:
# Librerías.
import pandas as pd
import Funciones as f

In [ ]:
%%capture
%run "6. Relleno de medianas de IP asociados.ipynb"

In [3]:
# Desfragmentar todos los DataFrames para mejorar el rendimiento.
for Nombre_df in dfs_Finales.keys():
    dfs_Finales[Nombre_df] = dfs_Finales[Nombre_df].copy()

In [4]:
# Invertimos los valores.
for Nombre_df, df in dfs_Finales.items():
   df[Items_Progresistas] = df[Items_Progresistas].apply(lambda x: 6 - pd.to_numeric(x, errors='coerce'))

In [5]:
# Convertir a float.
for Nombre_df, df in dfs_Finales.items():
   df[Items_Progresistas] = df[Items_Progresistas].astype(float)
   df[Items_Progresistas_Tiempo] = df[Items_Progresistas_Tiempo].astype(float)

In [6]:
# Calcular Indice de progresismo.
for Nombre_df, df in dfs_Finales.items():
   df['Indice_Progresismo'] = df[Items_Progresistas].mean(axis=1)

In [7]:
# Calcular Indice de tiempo en items de progresismo.
for Nombre_df, df in dfs_Finales.items():
   df['Indice_Progresismo_Tiempo'] = df[Items_Progresistas_Tiempo].mean(axis=1)

In [ ]:
print("="*80)
print("VERIFICACIÓN: ÍNDICE DE PROGRESISMO")
print("="*80)

# 1. Verificar existencia de variables necesarias.
print("1. VERIFICACIÓN DE VARIABLES NECESARIAS:")
Variables_Requeridas = ['Items_Progresistas', 'Items_Progresistas_Tiempo']

for Variable in Variables_Requeridas:
    if Variable in globals():
        Valor_Variable = globals()[Variable]
        print(f"   ✅ {Variable}: {len(Valor_Variable) if hasattr(Valor_Variable, '__len__') else 'Definida'}")
        if hasattr(Valor_Variable, '__len__') and len(Valor_Variable) > 0:
            print(f"      Contenido: {list(Valor_Variable)[:10]}...")  # Primeros 10 elementos.
    else:
        print(f"   ❌ {Variable}: NO ENCONTRADA")
        # Definir manualmente si no existe (ítems progresistas típicos).
        if Variable == 'Items_Progresistas':
            Items_Progresistas = [
                'IP_Item_5_Respuesta', 'IP_Item_6_Respuesta', 'IP_Item_9_Respuesta',
                'IP_Item_11_Respuesta', 'IP_Item_16_Respuesta', 'IP_Item_20_Respuesta',
                'IP_Item_24_Respuesta', 'IP_Item_25_Respuesta', 'IP_Item_27_Respuesta',
                'IP_Item_28_Respuesta'
            ]
            print(f"      🔧 Definiendo manualmente: {Items_Progresistas}")
        elif Variable == 'Items_Progresistas_Tiempo':
            Items_Progresistas_Tiempo = [
                'IP_Item_5_Tiempo', 'IP_Item_6_Tiempo', 'IP_Item_9_Tiempo',
                'IP_Item_11_Tiempo', 'IP_Item_16_Tiempo', 'IP_Item_20_Tiempo',
                'IP_Item_24_Tiempo', 'IP_Item_25_Tiempo', 'IP_Item_27_Tiempo',
                'IP_Item_28_Tiempo'
            ]
            print(f"      🔧 Definiendo manualmente: {Items_Progresistas_Tiempo}")

print("\n" + "="*60)

for Nombre_df, df in dfs_Finales.items():
    print(f"\n📊 DATAFRAME: {Nombre_df}")
    print("-" * 50)
    
    # 2. Verificar existencia de columnas de ítems progresistas.
    print("2. VERIFICACIÓN DE COLUMNAS PROGRESISTAS:")
    
    if 'Items_Progresistas' in globals():
        Columnas_Encontradas = []
        Columnas_Faltantes = []
        
        for Item in Items_Progresistas:
            if Item in df.columns:
                Columnas_Encontradas.append(Item)
            else:
                Columnas_Faltantes.append(Item)
        
        print(f"   ✅ Columnas encontradas: {len(Columnas_Encontradas)}")
        print(f"   ❌ Columnas faltantes: {len(Columnas_Faltantes)}")
        
        if Columnas_Faltantes:
            print(f"      Faltantes: {Columnas_Faltantes}")
    
    # 3. Verificar la inversión de escala.
    print("\n3. VERIFICACIÓN DE INVERSIÓN DE ESCALA:")
    
    if 'Items_Progresistas' in globals() and len(Columnas_Encontradas) > 0:
        # Tomar una muestra de la primera columna progresista.
        Primera_Columna = Columnas_Encontradas[0]
        
        # Verificar rango de valores después de la inversión.
        Valores_Unicos = df[Primera_Columna].dropna().unique()
        Valores_Unicos_Ordenados = sorted([v for v in Valores_Unicos if pd.notna(v)])
        
        print(f"   Muestra en {Primera_Columna}:")
        print(f"   Valores únicos después de inversión: {Valores_Unicos_Ordenados}")
        
        # Verificar si los valores están en el rango esperado [1-5].
        Valores_Fuera_Rango = [v for v in Valores_Unicos_Ordenados if v < 1 or v > 5]
        if Valores_Fuera_Rango:
            print(f"   ⚠️  Valores fuera de rango [1-5]: {Valores_Fuera_Rango}")
        else:
            print(f"   ✅ Todos los valores están en rango [1-5]")
        
        # Verificar tipos de datos.
        Tipo_Datos = str(df[Primera_Columna].dtype)
        print(f"   Tipo de datos: {Tipo_Datos}")
        
        if 'float' in Tipo_Datos:
            print("   ✅ Conversión a float exitosa")
        else:
            print("   ⚠️  No es tipo float")
    
    # 4. Verificar cálculo del Índice de Progresismo.
    print("\n4. VERIFICACIÓN DEL ÍNDICE DE PROGRESISMO:")
    
    if 'Indice_Progresismo' in df.columns:
        print("   ✅ Columna 'Indice_Progresismo' creada")
        
        # Estadísticas descriptivas.
        Estadisticas_Indice = df['Indice_Progresismo'].describe()
        print(f"   Estadísticas descriptivas:")
        print(f"     Count: {Estadisticas_Indice['count']:.0f}")
        print(f"     Mean: {Estadisticas_Indice['mean']:.3f}")
        print(f"     Std: {Estadisticas_Indice['std']:.3f}")
        print(f"     Min: {Estadisticas_Indice['min']:.3f}")
        print(f"     Max: {Estadisticas_Indice['max']:.3f}")
        
        # Verificar rango esperado.
        if Estadisticas_Indice['min'] >= 1 and Estadisticas_Indice['max'] <= 5:
            print("   ✅ Índice en rango esperado [1-5]")
        else:
            print(f"   ⚠️  Índice fuera de rango esperado: [{Estadisticas_Indice['min']:.3f}, {Estadisticas_Indice['max']:.3f}]")
        
        # Verificar valores NaN.
        Valores_NaN = df['Indice_Progresismo'].isna().sum()
        Total_Filas = len(df)
        Porcentaje_NaN = (Valores_NaN / Total_Filas) * 100
        print(f"   Valores NaN: {Valores_NaN} ({Porcentaje_NaN:.1f}%)")
        
        # Muestra de valores del índice.
        Muestra_Indices = df['Indice_Progresismo'].dropna().head(10).tolist()
        print(f"   Muestra de valores: {[round(v, 3) for v in Muestra_Indices]}")
        
    else:
        print("   ❌ Columna 'Indice_Progresismo' NO encontrada")
    
    # 5. Verificar Índice de Tiempo de Progresismo.
    print("\n5. VERIFICACIÓN DEL ÍNDICE DE TIEMPO:")
    
    if 'Indice_Progresismo_Tiempo' in df.columns:
        print("   ✅ Columna 'Indice_Progresismo_Tiempo' creada")
        
        # Estadísticas descriptivas.
        Estadisticas_Tiempo = df['Indice_Progresismo_Tiempo'].describe()
        print(f"   Estadísticas descriptivas:")
        print(f"     Count: {Estadisticas_Tiempo['count']:.0f}")
        print(f"     Mean: {Estadisticas_Tiempo['mean']:.3f}")
        print(f"     Std: {Estadisticas_Tiempo['std']:.3f}")
        print(f"     Min: {Estadisticas_Tiempo['min']:.3f}")
        print(f"     Max: {Estadisticas_Tiempo['max']:.3f}")
        
        # Verificar valores negativos (no deberían existir).
        Valores_Negativos = (df['Indice_Progresismo_Tiempo'] < 0).sum()
        if Valores_Negativos > 0:
            print(f"   ⚠️  Valores negativos: {Valores_Negativos}")
        else:
            print("   ✅ No hay valores negativos")
        
        # Valores NaN.
        Valores_NaN_Tiempo = df['Indice_Progresismo_Tiempo'].isna().sum()
        Porcentaje_NaN_Tiempo = (Valores_NaN_Tiempo / Total_Filas) * 100
        print(f"   Valores NaN: {Valores_NaN_Tiempo} ({Porcentaje_NaN_Tiempo:.1f}%)")
        
    else:
        print("   ❌ Columna 'Indice_Progresismo_Tiempo' NO encontrada")
    
    # 6. Verificación manual de cálculo de índice.
    print("\n6. VERIFICACIÓN MANUAL DEL CÁLCULO:")
    
    if ('Indice_Progresismo' in df.columns and 
        'Items_Progresistas' in globals() and 
        len(Columnas_Encontradas) > 0):
        
        # Tomar una fila aleatoria para verificación manual.
        Fila_Muestra = df.dropna(subset=['Indice_Progresismo']).iloc[0]
        ID_Muestra = Fila_Muestra['ID']
        Indice_Calculado = Fila_Muestra['Indice_Progresismo']
        
        print(f"   Verificando ID: {ID_Muestra}")
        print(f"   Índice calculado: {Indice_Calculado:.4f}")
        
        # Calcular manualmente.
        Valores_Items = []
        for Item in Columnas_Encontradas:
            Valor = Fila_Muestra[Item]
            if pd.notna(Valor):
                Valores_Items.append(Valor)
                print(f"     {Item}: {Valor}")
        
        if len(Valores_Items) > 0:
            Indice_Manual = sum(Valores_Items) / len(Valores_Items)
            print(f"   Índice manual: {Indice_Manual:.4f}")
            
            Diferencia = abs(Indice_Calculado - Indice_Manual)
            if Diferencia < 0.001:
                print("   ✅ Cálculo correcto")
            else:
                print(f"   ⚠️  Diferencia: {Diferencia:.4f}")
    
    # 7. Distribución por categorías.
    print("\n7. DISTRIBUCIÓN POR CATEGORÍAS PASO:")
    
    if ('Indice_Progresismo' in df.columns and 
        'Categoria_PASO_2023' in df.columns):
        
        Distribucion_Categorias = df.groupby('Categoria_PASO_2023')['Indice_Progresismo'].agg(['count', 'mean', 'std'])
        
        print("   Categoría               | Count | Mean  | Std   |")
        print("   " + "-"*50)
        
        for Categoria, Stats in Distribucion_Categorias.iterrows():
            if pd.notna(Stats['mean']):
                print(f"   {Categoria:<22} | {Stats['count']:5.0f} | {Stats['mean']:5.2f} | {Stats['std']:5.2f} |")

print("\n" + "="*80)
print("✅ VERIFICACIÓN DEL ÍNDICE DE PROGRESISMO COMPLETADA")
print("="*80)

# 8. Verificación adicional: Coherencia de la inversión.
print("\n" + "="*80)
print("VERIFICACIÓN ADICIONAL: COHERENCIA DE LA INVERSIÓN")
print("="*80)

for Nombre_df, df in dfs_Finales.items():
    print(f"\n🔍 DATAFRAME: {Nombre_df}")
    print("-" * 40)
    
    if 'Items_Progresistas' in globals():
        print("Verificando la lógica de inversión (6 - valor):")
        
        # Crear tabla de verificación para los primeros casos.
        if len(Columnas_Encontradas) > 0:
            Primera_Columna = Columnas_Encontradas[0]
            
            print(f"\nTabla de verificación para {Primera_Columna}:")
            print("Original | Invertido | Esperado")
            print("-" * 30)
            
            # Mostrar ejemplos de la transformación.
            Casos_Verificacion = []
            for Valor_Original in [1, 2, 3, 4, 5]:
                Valor_Esperado = 6 - Valor_Original
                
                # Buscar casos reales con este valor.
                # Como ya está invertido, buscamos el valor esperado.
                Casos_Reales = df[df[Primera_Columna] == Valor_Esperado]
                
                if len(Casos_Reales) > 0:
                    print(f"    {Valor_Original}    |     {Valor_Esperado}     |     {Valor_Esperado}")
                    Casos_Verificacion.append((Valor_Original, Valor_Esperado, Valor_Esperado))
            
            print(f"\n✅ Transformación verificada para {len(Casos_Verificacion)} valores")

print("\n" + "="*80)
print("✅ VERIFICACIÓN DE COHERENCIA COMPLETADA")
print("="*80)

In [9]:
# Exportar base para el control.
for Nombre_df, df in dfs_Finales.items():
    dfs_Finales[Nombre_df].head(50).to_excel(f'Controles/7. Indice de progresismo ({Nombre_df}).xlsx', index=False)